In [1]:
import sys
!conda install --yes --prefix {sys.prefix} pandas
!conda install --yes --prefix {sys.prefix} networkx
!conda install --yes --prefix {sys.prefix} matplotlib

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\lampu\miniconda3\envs\annotations

  added / updated specs:
    - pandas


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2022.9.24          |  py310haa95532_0         155 KB
    pandas-1.4.4               |  py310hd77b12b_0         8.7 MB
    ------------------------------------------------------------
                                           Total:         8.8 MB

The following packages will be UPDATED:



==> WARNING: A newer version of conda exists. <==
  current version: 4.13.0
  latest version: 22.9.0

Please update conda by running

    $ conda update -n base -c defaults conda






  ca-certificates                      2022.4.26-haa95532_0 --> 2022.07.19-haa95532_0
  certifi                       2022.5.18.1-py310haa95532_0 --> 2022.9.24-py310haa95532_0
  openssl                                 1.1.1o-h2bbff1b_0 --> 1.1.1q-h2bbff1b_0
  pandas                              1.4.2-py310hd77b12b_0 --> 1.4.4-py310hd77b12b_0




certifi-2022.9.24    | 155 KB    |            |   0% 
certifi-2022.9.24    | 155 KB    | ########## | 100% 
certifi-2022.9.24    | 155 KB    | ########## | 100% 

pandas-1.4.4         | 8.7 MB    |            |   0% 
pandas-1.4.4         | 8.7 MB    | 4          |   5% 
pandas-1.4.4         | 8.7 MB    | #6         |  17% 
pandas-1.4.4         | 8.7 MB    | ##9        |  30% 
pandas-1.4.4         | 8.7 MB    | ####2      |  43% 
pandas-1.4.4         | 8.7 MB    | #####7     |  57% 
pandas-1.4.4         | 8.7 MB    | ######9    |  69% 
pandas-1.4.4         | 8.7 MB    | ########2  |  83% 
pandas-1.4.4         | 8.7 MB    | #########8 |  98% 
p



==> WARNING: A newer version of conda exists. <==
  current version: 4.13.0
  latest version: 22.9.0

Please update conda by running

    $ conda update -n base -c defaults conda




Solving environment: ...working... done

# All requested packages already installed.





==> WARNING: A newer version of conda exists. <==
  current version: 4.13.0
  latest version: 22.9.0

Please update conda by running

    $ conda update -n base -c defaults conda




# 1) Import JSON into DataFrame

Export annotations from CVAT as COCO JSON. 

Put the json file in the working directory and change the variable "filename".

In [5]:
import pandas as pd
import json

# utilities

def findCategory(data):
    # find categories
    cats = data["categories"]
    category = pd.DataFrame(cats)
    category = category.drop(['supercategory'], axis=1)
    category = category.rename(columns={'id': 'category_id'})
    return category

def findImages(data):
    img = data["images"]
    images = pd.DataFrame(img)
    
    # unwanted columns exist if exported from CVAT. Not if generated by my code
    if set(['license','flickr_url','coco_url','date_captured']).issubset(images.columns):
        images = images.drop(columns=['license','flickr_url','coco_url','date_captured'])
    
    return images

def findAnnotations(data):
    anno = data["annotations"]
    df = pd.DataFrame(anno)
    return df

# convert all np.integer, np.floating and np.ndarray into json recognisable int, float and lists
class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)

In [6]:
import pandas as pd
import numpy as np
import re
import json

df = pd.DataFrame()
# area = 'A14 Tothill'
filename = './input/combined.json'
with open(filename, 'r') as file:
    data = json.load(file)
    
    category = findCategory(data)

    images = findImages(data)
    nos_image = images['id'].max()

    df = findAnnotations(data)


In case the old CVAT xml is pursued...

In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import re
import json

df = pd.DataFrame()
area = 'A14 Tothill'
filename = 'train_221222.csv'
# filename = 'new_'+ area +'.csv'
with open(filename, 'r', encoding='utf-8-sig', errors="surrogateescape") as file:
    for i, line in enumerate(file,1):
        dict_train = eval(line)
        add = pd.DataFrame.from_dict(dict_train, orient='index')
        add = add.transpose()
        if i == 1:
            df = df.append(dict_train, ignore_index = True)
        else:
            df = pd.concat([df, add], ignore_index = True)

        # df = df.append(dict_train, ignore_index = True)

# # converting json dataset from dictionary to dataframe
# train = pd.DataFrame.from_dict(dict_train, orient='index')
# train.reset_index(level=0, inplace=True)

df

C:\Users\lampu\AppData\Local\Temp\ipykernel_20512\2225267564.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(dict_train, ignore_index = True)


,id,image_id,category_id,segmentation,area,bbox,iscrowd,attributes
0,1,5,4,"[[663.0, 9.31, 732.46, 957.05, 771.05, 963.22,...",31932.0,"[663.0, 9.31, 108.05, 953.91]",0,{'occluded': False}
1,2,6,4,"[[669.18, 13.94, 761.79, 907.66, 789.57, 904.5...",22183.0,"[669.18, 13.94, 120.39, 893.72]",0,{'occluded': False}
2,3,7,4,"[[679.98, 13.94, 797.29, 1018.79, 831.25, 1021...",29303.0,"[679.98, 13.94, 151.27, 1007.94]",0,{'occluded': False}
3,4,8,4,"[[690.79, 7.77, 820.45, 987.92, 863.66, 995.64...",33488.0,"[690.79, 7.77, 172.87, 987.87]",0,{'occluded': False}
4,5,9,4,"[[693.87, 13.94, 845.14, 901.48, 877.56, 896.8...",27143.0,"[693.87, 12.4, 183.69, 889.08]",0,{'occluded': False}
...,...,...,...,...,...,...,...,...
1812,1813,2328,4,"[[718.57, 6.23, 754.07, 47.9, 794.2, 89.58, 82...",18234.0,"[718.57, 3.14, 327.23, 395.15]",0,{'occluded': False}
1813,1814,2328,11,"[[84.11, 185.55, 145.96, 184.31, 174.42, 145.9...",2652.0,"[84.11, 139.78, 90.31, 45.77]",0,{'occluded': False}
1814,1815,2328,11,"[[1044.07, 132.36, 1056.44, 170.71, 1131.9, 16...",3279.0,"[1044.07, 118.75, 87.83, 51.96]",0,{'occluded': False}
1815,1816,2329,4,"[[806.55, 38.64, 854.4, 34.01, 880.64, 64.88, ...",7664.0,"[806.55, 34.01, 422.93, 290.19]",0,{'occluded': False}


## 2a) Query numbers and images of a certain category

In [4]:
# Query category_id
defect = 'potholes'

# match the defect with the category df
# the extraction outputs a series. use [0] to get the value
q_catid = category[category['name']==defect]['category_id'].values[0]

# Excerpt df
ans = df[df['category_id']==q_catid]
# add file_name
ans = ans.merge(images[['id','file_name']], left_on='image_id', right_on='id')
ans = ans.drop(columns=['iscrowd','attributes','id_y'])

# Export CSV of images the defect
csv_need = False
if csv_need == True:
    name = defect+'_image.csv'
    ans.to_csv(name)


ans

,id_x,image_id,category_id,segmentation,area,bbox,file_name
0,293,1635,10,"[[1620.8, 529.2, 1616.9, 532.9, 1601.6, 532.9,...",19439.0,"[1574.7, 529.2, 180.9, 124.6]",A12Area6SwallowsCrossMountnessingHuttonBrentwo...


## 2b) Move files included in JSON

Useful in cases like after spliting a training and testing set with cocosplit, input the testing set and copy the images into another directory

In [44]:
import glob, shutil, os

IMAGE_DIR = '../data/train/images/'
# IMAGE_DIR = 'D:\\04 Trained/230110/images/train/'
DST_DIR = './output_img/'

# filter the images with defects in "annotations"
ans = df
ans = ans.merge(images[['id','file_name']], left_on='image_id', right_on='id')
ans = ans.drop(columns=['iscrowd','attributes','id_y'])
concerned_img = ans['file_name'].unique().tolist()

# copy the concerned images
for i in range(len(concerned_img)):
    IMAGE_PATHS = glob.glob(os.path.join(IMAGE_DIR, concerned_img[i]))
    try:
        shutil.copy(IMAGE_PATHS[0],DST_DIR)
    except:
        print(concerned_img[i],' not found.')
        continue

# 2) Count numbers of defects

In [7]:
# Count total number of images
findTotal = False
if findTotal == True:
    if area == 'A14 Tothill':
        nos_image = 2343
    elif area == 'A12 Mountnessing':
        nos_image = 6580
# if not imported from json, manually input nos. images
# nos_image = [self defined]
        
# Count images with defects by querying image id
qall = True
if qall == True:
    start_id = 1
    end_id = max(df['image_id'])
elif qall == False:
    start_id = 4356
    end_id = 6580

df2 = df[(df['image_id']>=start_id) & (df['image_id']<=end_id)]

nos_image_d = df2['image_id'].nunique()

nos_defects = len(df2['id'])

print(f'{nos_image_d} out of {nos_image} images have defects.\n{nos_defects} defects in total')

defect_cat = pd.pivot_table(df2, values='id', index='category_id', aggfunc='count')

defect_cat = defect_cat.merge(category[['category_id','name']], left_on='category_id', right_on='category_id')
defect_cat = defect_cat.rename(columns={'id': 'counts'})
defect_cat

641 out of 641 images have defects.
1451 defects in total


,category_id,counts,name
0,2,177,raveling
1,3,290,crack_transverse
2,4,286,crack_longitudinal
3,5,94,crack_edge
4,10,45,potholes
5,11,270,patch
6,12,289,unknown


# 3) Create YOLO Label Files from COCO JSON

In [32]:
import re
import yaml

# For training without specific categories
cat_elim = [4, 11]

# find if image_id has any defects
for i in images['id']:
    if (i in df['image_id'].unique()) == True:
        # print(f'image {i} has defect')
        
        # Concatenate df and images for the text file
        df2 = df[df['image_id']==i]
        image2 = images.loc[(images['id']==i),['id','width','height','file_name']]
        image2 = image2.rename(columns={'id': 'image_id'})
        df2 = df2.drop(columns=['area','bbox','iscrowd','attributes'])
        df2 = df2.merge(image2,left_on='image_id',right_on='image_id') 
        
        # Normalise polygon points
        poly_print = []
        im_width = df2['width'][0]
        im_height = df2['height'][0]
        for f, x in enumerate(df2['segmentation'].to_list()):
            if df2['category_id'][f] in cat_elim:
                # skip unwanted categories
                continue
            else:
                poly = [df2['category_id'][f]-1]
                # print(f'image {i} polygon length {len(x[0])}')
                for y in range(1,len(x[0]),2):
                    # print(x,y)
                    ptx = x[0][y-1]/im_width
                    pty = x[0][y]/im_height
                    poly.extend([ptx, pty])
                # poly.extend([poly[1],poly[2]])                      # YOLO does not need polygon points to form a close loop
                poly_print.append(poly)
                poly = []
        
        fname = re.findall(r'(.*)(?:.jpg)$',df2['file_name'][0])[0]
        fname = "./txt_labels/"+fname+".txt"
        with open(fname, "w") as outfile:
            for num, j in enumerate(poly_print):
                outfile.write(' '.join(str(e) for e in j))
                # print(f'image {i} defect {num} of {len(poly_print)}')
                if num==len(poly_print)-1: outfile.write('')
                else: outfile.write('\n')
        poly_print = []
    else:
        fname = re.findall(r'(.*)(?:.jpg)$',images.loc[(images['id']==i),'file_name'].values[0])[0]
        fname = "./txt_labels/"+fname+".txt"
        with open(fname, "w") as outfile:
            outfile.write('')

print(f'Labels of {nos_image} images prepared!')

# Create data.yaml file
nc = len(category['category_id'])
names = category['name'].to_list()
with open(r'./custom.yaml', "w") as outfile:
    outfile.write('train: ./data/train/images \n')
    outfile.write('val: ./data/val/images \n')
    outfile.write('test: ./data/test/images \n \n')
    outfile.write(f'nc: {nc} \n')
    outfile.write(f'names: {names} \n')
print('custom.yaml prepared!')

# np.array(df2['segmentation'].to_list()[0])/10

Labels of 5882 images prepared!
custom.yaml prepared!


## Instructions

Now move the folder "txt_labels" to the corresponding ./data/[test/train/val] folder

Move also a the custom.yaml to ./data

Leave the folder "txt_labels" in this ./fix_annotation folder for subsequent annotation files.

Happy training!

# 4) Convert YOLO Labels to COCO JSON

In [2]:
import glob, os
import yaml
import pandas as pd
import numpy as np
import re
import json
from PIL import Image
# from imantics import Mask
from shapely.geometry import Polygon

IMAGE_DIR = '../data/test/images/'
# IMAGE_DIR = os.path.join(os.getcwd(), '..' ) + '/data/test/images/'
IMAGE_PATHS = glob.glob(os.path.join(IMAGE_DIR, '*.jp*g'))
LABEL_DIR = './input/'
LABEL_PATHS = glob.glob(os.path.join(LABEL_DIR, '*.txt'))
# SAVE_DIR = './labels/test_annotated/'

with open("custom.yaml", "r") as cat:
    data = yaml.safe_load(cat)
    category = pd.DataFrame(data['names'], columns=['name'])
    category['id'] = category.index + 1
    category['supercategory'] = ""
    category = category[['id','name','supercategory']]


for image_id, image_path in enumerate(IMAGE_PATHS):
    img = Image.open(image_path)
    if image_id == 0:
        image = pd.DataFrame(columns=['id','width','height','file_name'])

    add = dict()
    add['id'] = image_id + 1
    add['width'], add['height'] = img.size
    # add['file_name']= img.filename.split("/")[-1]
    add['file_name'] = re.findall(r'^.*\\([^\/]*)$',img.filename)[-1]
    add_im = pd.DataFrame(add, index=[0])
    image = pd.concat([image, add_im], ignore_index = True)   



for txt_id, txt_path in enumerate(LABEL_PATHS):
    if txt_id == 0:
        df = pd.DataFrame(columns=['id','image_id','category_id','segmentation','area','bbox','iscrowd', 'attributes'])
        full_anno = pd.DataFrame()
    
    with open(txt_path, 'r', encoding='utf-8-sig', errors="surrogateescape") as file:
        # if df.shape[0]==0:
        if len(df['id']) == 0:
            prev_defect = 0
        else:
            prev_defect = max(df['id'])
        
        for i, line in enumerate(file):
            inner_list = [float(elt.strip()) for elt in line.split(' ')]
            defect = dict()
            
            filename = re.findall(r'^.*\\([^\/]*)(?:s|_mask.txt)$',txt_path)[-1] + ".jpg"
            # filename = re.findall(r'^.*\\([^\/]*)(?:s|.txt)$',txt_path)[-1] + ".jpg"
            defect['id'] = prev_defect + i + 1
            defect['image_id'] = image.loc[(image['file_name']==filename), 'id'].values[0]
            # defect['image_id'] = txt_id + 1
            defect['category_id'] = int(inner_list[0]) + 1
            
            # upscale segmentation
            seg = np.array(inner_list[1:])
            x_scale = image.loc[(image['id']==defect['image_id']), 'width'].values      # x width
            y_scale = image.loc[(image['id']==defect['image_id']), 'height'].values     # y height
            # print(x_scale, y_scale)
            for y in range(1,len(seg),2):
                seg[y] = round(float(seg[y] * y_scale),1)
                seg[y-1] = round(float(seg[y-1] * x_scale),1)    
            seg_concat = [[]]
            seg_concat[0][:] = seg.tolist()
            defect['segmentation'] = seg_concat
            # defect['segmentation'] = seg.to_list()

            # Find bbox
            x_min, x_max = min(seg[::2]), max(seg[::2])
            y_min, y_max = min(seg[1::2]), max(seg[1::2])
            defect['bbox'] = [x_min, y_min, round(x_max-x_min,1), round(y_max-y_min,1)]
            # Find area
            coord = np.transpose(np.array([seg[::2],seg[1::2]])) # Polygon work with dimensions (n, 2)
            try:
                defect['area'] = round(Polygon(coord).area,1)
            except:
                print('defect id: {}, image: {}'.format(defect['id'], image.loc[(image['id']==defect['image_id']), 'file_name'].values))
                continue
            defect['iscrowd'] = 0
            defect['attributes'] = {"occluded":False}

            add_df = pd.DataFrame([defect], index=[0])
            df = pd.concat([df, add_df], ignore_index = True)


# Write categories, images and annotations as arrays containing individual entries as a dictionary
# see https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_dict.html for to_dict styles
dict_to_json = {
    "categories": category.to_dict('records'),
    "images": image.to_dict('records'),
    "annotations": df.to_dict('records')
    }

# convert all np.integer, np.floating and np.ndarray into json recognisable int, float and lists
class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)

with open("./inferred.json", "w") as outfile:
    json.dump(dict_to_json, outfile, cls=NpEncoder)

df

defect id: 139, image: ['A12Area6SwallowsCrossMountnessingHuttonBrentwoodLondon_sideview_000020_003239_Lane2.jpg']
defect id: 148, image: ['A12Area6SwallowsCrossMountnessingHuttonBrentwoodLondon_sideview_000020_003239_Lane2.jpg']
defect id: 162, image: ['A12Area6SwallowsCrossMountnessingHuttonBrentwoodLondon_sideview_000020_003242_Lane2.jpg']
defect id: 187, image: ['A12Area6SwallowsCrossMountnessingHuttonBrentwoodLondon_sideview_000020_003357_Lane2.jpg']
defect id: 226, image: ['A12Area6SwallowsCrossMountnessingHuttonBrentwoodLondon_sideview_000023_000238_Lane1.jpg']
defect id: 277, image: ['A12Area6SwallowsCrossMountnessingHuttonBrentwoodLondon_sideview_000023_000345_Lane1.jpg']
defect id: 307, image: ['A12Area6SwallowsCrossMountnessingHuttonBrentwoodLondon_sideview_000023_000392_Lane1.jpg']
defect id: 387, image: ['A12Area6SwallowsCrossMountnessingHuttonBrentwoodLondon_sideview_000023_000671_Lane1.jpg']
defect id: 564, image: ['A12Area6SwallowsCrossMountnessingHuttonBrentwoodLondon_

,id,image_id,category_id,segmentation,area,bbox,iscrowd,attributes
0,1,1,11,"[[2410.0, 638.8, 2406.3, 642.5, 2406.3, 653.8,...",3226.7,"[2402.4, 638.8, 57.7, 64.1]",0,{'occluded': False}
1,2,2,11,"[[2329.2, 393.1, 2325.3, 396.8, 2302.4, 396.8,...",17230.7,"[2294.7, 393.1, 165.4, 147.4]",0,{'occluded': False}
2,3,2,11,"[[223.2, 362.9, 219.5, 366.6, 188.7, 366.6, 18...",11394.8,"[142.4, 362.9, 146.4, 98.3]",0,{'occluded': False}
3,4,4,11,"[[950.9, 75.7, 947.2, 79.4, 931.6, 79.4, 927.9...",12392.9,"[924.0, 75.7, 219.5, 64.1]",0,{'occluded': False}
4,5,5,11,"[[308.0, 211.6, 304.1, 215.5, 296.4, 215.5, 28...",17849.3,"[246.4, 211.6, 219.5, 98.2]",0,{'occluded': False}
...,...,...,...,...,...,...,...,...
714,726,850,11,"[[169.5, 302.4, 165.6, 306.1, 138.7, 306.1, 13...",29376.9,"[76.9, 302.4, 396.7, 192.7]",0,{'occluded': False}
715,727,853,11,"[[365.7, 166.3, 362.0, 170.0, 288.8, 170.0, 28...",25299.0,"[215.6, 166.3, 358.0, 162.5]",0,{'occluded': False}
716,728,854,11,"[[111.6, 393.1, 107.7, 396.8, 104.0, 396.8, 10...",57148.7,"[0.0, 393.1, 512.0, 222.9]",0,{'occluded': False}
717,729,855,11,"[[585.2, 151.1, 581.3, 155.0, 362.0, 155.0, 35...",22138.1,"[261.7, 151.1, 389.0, 128.5]",0,{'occluded': False}


# 5) Combine COCO JSON

Useful when patches and crack_longitudinal are detected using one trained model, while other defects are detected with another

In [4]:
import pandas as pd
import numpy as np
import re
import json

# Files to be combined
filename = './input/inferred_withpatch.json'
filename2 = './input/inferred_nopatch.json'

# Store categories, images and annotations in separate dataframes
with open(filename, 'r') as file:
    data = json.load(file)
    
    category = findCategory(data)
    images = findImages(data)
    nos_image = images['id'].max()
    df = findAnnotations(data)
    df = df.merge(images[['id','file_name']], left_on='image_id', right_on='id')
    df = df.rename(columns={'id_x': 'id'})
    df = df.drop(columns=['iscrowd','attributes','id_y'])

with open(filename2, 'r') as file:
    data2 = json.load(file)
    
    category2 = findCategory(data2)
    images2 = findImages(data2)
    nos_image2 = images2['id'].max()
    df2 = findAnnotations(data2)
    df2 = df2.merge(images2[['id','file_name']], left_on='image_id', right_on='id')
    df2 = df2.rename(columns={'id_x': 'id'})
    df2 = df2.drop(columns=['iscrowd','attributes','id_y'])

# Categories - Check if categories are the same
# check length
if len(category) != len(category2):
    print('categories not the same. Check before proceeding')
# check each category
for i in range(len(category['name'])):
    if category['name'][i] != category2['name'][i]:
        print('category id: {} , {} in file 1 different from category id: {} , {} in file 2. Please check'.format(category['category_id'][i], category['name'][i], category2['category_id'][i], category2['name'][i]))
# clean category for json dump
category = category.rename(columns={'category_id': 'id'})
category['supercategory'] = ""

# Annotations
# merge two annotation df
df_new = pd.concat([df, df2], ignore_index = True)

# combine and re-arrange the image info
images_new = pd.DataFrame(columns=['id','width','height','file_name'])
images_new['file_name'] = df_new['file_name'].unique()
images_new = images_new.sort_values(by=['file_name'], ignore_index=True)
images_new['id'] = images_new.index + 1
for i in range(len(images_new['file_name'])):
    # find out which json records the concerned image
    if images_new['file_name'][i] in images['file_name'].tolist():
        dim = images.loc[(images['file_name']==images_new['file_name'][i]),['width','height']]
    elif images_new['file_name'][i] in images2['file_name'].tolist():
        dim = images2.loc[(images2['file_name']==images_new['file_name'][i]),['width','height']]
    else:
        print('image not included')
        continue
    # paste width and height info
    images_new.loc[i,'width'] = dim['width'].values[0]
    images_new.loc[i,'height'] = dim['height'].values[0]


# assign new image id and defect id in df_new
for i in range(len(df_new['id'])):
    df_new.loc[i, 'image_id'] = images_new.loc[(images_new['file_name']==df_new['file_name'][i]), 'id'].values
df_new = df_new.sort_values(by=['image_id'], ignore_index=True)
df_new['id'] = df_new.index + 1
df_new['iscrowd'] = 0
df_new['attributes'] = [{'occluded':False}] * len(df_new['id'])
df_new = df_new.drop(columns=['file_name'])

# Write categories, images and annotations as arrays containing individual entries as a dictionary
# see https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_dict.html for to_dict styles
dict_to_json = {
    "categories": category.to_dict('records'),
    "images": images_new.to_dict('records'),
    "annotations": df_new.to_dict('records')
    }

with open("./combined.json", "w") as outfile:
    json.dump(dict_to_json, outfile, cls=NpEncoder)


# df_new
# print('image: {}. W{} * H{}'.format(images_new['file_name'][10], images_new['width'][10],images_new['height'][10]))

## Dataframe formats for reference

In [12]:
images


,id,width,height,file_name
0,1,2464,2056,A12Area6SwallowsCrossMountnessingHuttonBrentwo...
1,2,2464,2056,A12Area6SwallowsCrossMountnessingHuttonBrentwo...
2,3,2464,2056,A12Area6SwallowsCrossMountnessingHuttonBrentwo...
3,4,2464,2056,A12Area6SwallowsCrossMountnessingHuttonBrentwo...
4,5,2464,2056,A12Area6SwallowsCrossMountnessingHuttonBrentwo...
...,...,...,...,...
995,996,2464,2056,A12Area6SwallowsCrossMountnessingHuttonBrentwo...
996,997,2464,2056,A12Area6SwallowsCrossMountnessingHuttonBrentwo...
997,998,2464,2056,A12Area6SwallowsCrossMountnessingHuttonBrentwo...
998,999,2464,2056,A12Area6SwallowsCrossMountnessingHuttonBrentwo...


In [8]:
category

,category_id,name
0,1,bleeding
1,2,raveling
2,3,crack_transverse
3,4,crack_longitudinal
4,5,crack_edge
5,6,crack_alligator
6,7,crack_block
7,8,shoving
8,9,rutting
9,10,potholes
